In [1]:
# %pip install adversarial-robustness-toolbox

In [2]:
import warnings
warnings.filterwarnings('ignore')
import os
import time

import keras

from keras.datasets import cifar10

from art.attacks import FastGradientMethod, ProjectedGradientDescent
from art.classifiers import KerasClassifier
from art.utils import load_dataset
from art.attacks import BasicIterativeMethod
from art.defences import AdversarialTrainer
from art.data_generators import KerasDataGenerator

from keras.datasets import cifar10
from keras.optimizers import adam
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.models import Sequential,Model,load_model
from keras import applications

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

from keras.optimizers import SGD, Adam
import numpy as np

from keras.callbacks import LearningRateScheduler
from math import floor
from math import ceil

import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import matplotlib.pyplot as plt

from tensorflow.python.keras.utils.data_utils import Sequence


# print(keras.__version__)

Using TensorFlow backend.


In [3]:
(x_train, y_train), (x_test, y_test), min_, max_ = load_dataset(str('cifar10'))
# (x_train1, y_train1), (x_test1, y_test1) = cifar10.load_data() # difference is only one hot y vector?

In [4]:
TEST_RUN = False
USE_PARTIAL_DATA = False

if TEST_RUN:
    USE_PARTIAL_DATA = True # how much of data we use for training
PARTIAL_SIZE = 256*8
img_size=32

USE_WEIGHT_DECAY = True

SHOW_DATA_AUG = False

USE_RETRAINING = False   # Either use retrain or train directly using PGD

VERBOSITY = 0  # Keras fit verbosity

In [5]:
class lr_schedule:
    def __init__(self, lr_init=0.01, factor=0.1, schedule=(10000, 10000)):
        self.schedule = schedule
        self.lr_init = lr_init
        self.factor = factor
    
    def __call__(self, epoch):
        exp = np.heaviside(epoch-self.schedule[0], 1) + np.heaviside(epoch-self.schedule[1], 1)
        return self.lr_init * self.factor ** exp

In [6]:
callbacks = []

epochs = 2

if USE_RETRAINING:
    epochs = 10
    batch_size = 256
    momentum = 0.9
    weight_decay = 5e-4
    
    retraining_rounds = 40  # Affects time the most, but probably the most important part
    epochs_per_round = 1  # More than 2 here is unnecessary I think
    
    lr = 0.0001 # lr = 0.01
    optimizer = Adam(lr=lr) # keras.optimizers.SGD(lr=lr, momentum=momentum) # Adam(lr=lr)
    loss = 'categorical_crossentropy'
    metrics = ['accuracy']
    
else:
    # trying to emulate their exact setup
    epochs1 = 150
    epochs2 = 100
    epochs3 = 100
    initial_epoch = 0
    batch_size = 256
    weight_decay = 5e-4
    
    momentum = 0.9
    lr1 = 0.01
    lr2 = 0.001
    lr3 = 0.0001
    # scheduler = lr_schedule(lr_init=lr, factor=0.1, schedule=(20, 35))
    
    # lr-schedule
    # callbacks = [LearningRateScheduler(scheduler)]
    
    # Data augmentation
    datagen = ImageDataGenerator(
                horizontal_flip=True,
                width_shift_range=0.1,
                height_shift_range=0.1,
                zoom_range=0.1,
                fill_mode='nearest')
    train_generator = datagen.flow(x=x_train, y=y_train, batch_size=batch_size)
    
    optimizer = keras.optimizers.SGD(lr=lr1, momentum=momentum, decay=0.0)
    loss = 'categorical_crossentropy'
    metrics = ['accuracy']
    
    adv_ratio = 1.0
    
#Adv training
norm = 2
eps = 0.5
steps = 7
step_size = 0.1
targeted = False

# MP
use_multiprocessing = False

# Checkpoints
checkpoint_interval = 10
checkpoint_interval *= ceil(x_train.shape[0] / batch_size)
print(checkpoint_interval)

filepath="impro-{epoch:02d}-{acc:.2f}.hdf5"
# checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')
checkpoint = ModelCheckpoint(filepath, period=checkpoint_interval, monitor='acc')
callbacks.append(checkpoint)

1960


In [7]:
input_shape = x_train.shape[1:]
no_classes = y_train.shape[1]

In [8]:
model = applications.resnet50.ResNet50(weights=None, include_top=True, input_shape=input_shape, classes=no_classes)

In [9]:
if SHOW_DATA_AUG:
    test_datagen = ImageDataGenerator(
                    # samplewise_std_normalization=True,
                    horizontal_flip=True,
                    width_shift_range=0.1,
                    height_shift_range=0.1,
                    zoom_range=0.1,
                    fill_mode='constant',
                    cval=0.5)
    test_gen = test_datagen.flow(x=x_train, y=y_train, batch_size=1)
    count = 0
    for img in test_gen:
        plt.imshow(img[0][0])
        plt.show()
        count += 1
        if count > 10:
            break

In [10]:
if USE_WEIGHT_DECAY:   # https://jricheimer.github.io/keras/2019/02/06/keras-hack-1/
    for layer in model.layers:
        if isinstance(layer, keras.layers.Conv2D) or isinstance(layer, keras.layers.Dense):
            layer.add_loss(keras.regularizers.l2(weight_decay)(layer.kernel))
        if hasattr(layer, 'bias_regularizer') and layer.use_bias:
            layer.add_loss(keras.regularizers.l2(weight_decay)(layer.bias))

In [11]:
if USE_PARTIAL_DATA:
    x_train, y_train = x_train[:PARTIAL_SIZE], y_train[:PARTIAL_SIZE]
    x_test, y_test = x_test[:PARTIAL_SIZE], y_test[:PARTIAL_SIZE]

In [12]:
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
classifier = KerasClassifier(model=model, clip_values=(min_, max_))

In [13]:
attack = ProjectedGradientDescent(classifier=classifier, norm=norm, eps=eps, eps_step=step_size, max_iter=steps, targeted=targeted, batch_size=batch_size)

In [14]:
test_size = 500
print("BEFORE TRAINING:")
predictions = classifier.predict(x_test[:test_size])
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test[:test_size], axis=1)) / len(y_test[:test_size])
print('Accuracy on benign test examples: {:.2f}%'.format(accuracy * 100))

x_test_adv = attack.generate(x=x_test[:test_size])
predictions = classifier.predict(x_test_adv)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test[:test_size], axis=1)) / len(y_test[:test_size])
print('Accuracy on adversarial test examples: {:.2f}%'.format(accuracy * 100))

BEFORE TRAINING:
Accuracy on benign test examples: 11.20%
Accuracy on adversarial test examples: 13.20%


In [15]:
# Saving trained model here
path = os.getcwd()
print(path)

/home/jupyter/CIFAR10


In [16]:
tf.get_logger().setLevel('INFO')
if TEST_RUN:
    generator = KerasDataGenerator(train_generator, x_train.shape[0], batch_size)
    trainer = AdversarialTrainer(classifier, attack, ratio=adv_ratio)
    trainer.fit_generator(generator=generator, nb_epochs=epochs, verbose=VERBOSITY, callbacks=callbacks, initial_epoch=initial_epoch, use_multiprocessing=use_multiprocessing)  
    
elif USE_RETRAINING:
    # Initial training
    start_time = time.time()
    classifier.fit(x_train, y_train, nb_epochs=epochs, batch_size=batch_size, verbose=VERBOSITY)
    
    file_name = 'robust_cifar10_init.h5'
    classifier.save(filename=file_name, path=path)
    print("Pretraining took {:.2f}s.".format(time.time() - start_time))
    
    print("MID TRAINING:")
    predictions = classifier.predict(x_test[:test_size])
    accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test[:test_size], axis=1)) / len(y_test[:test_size])
    print('Accuracy on benign test examples: {:.2f}%'.format(accuracy * 100))
    predictions = classifier.predict(x_test_adv)
    accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test[:test_size], axis=1)) / len(y_test[:test_size])
    print('Accuracy on adversarial test examples: {:.2f}%'.format(accuracy * 100))
    
    # Retraining for adversarial examples
    start_time_re = time.time()
    for i in range(retraining_rounds):
        print("Retraining round {}/{}".format(i+1, retraining_rounds))
        # Generating adv examples
        
        start_time = time.time()
        attack = ProjectedGradientDescent(classifier=classifier, norm=norm, eps=eps, eps_step=step_size, max_iter=steps, targeted=targeted, batch_size=batch_size)
        adv_x_train = attack.generate(x=x_train, y=y_train)
        print("Generating adversarial examples took {:.2f}s.".format(time.time() - start_time))
        
        # Retraining, a few fewer epochs I guess?
        classifier.fit(adv_x_train, y_train, nb_epochs=epochs_per_round, batch_size=batch_size, verbose=VERBOSITY)
        
        file_name = 'robust_cifar10_re' + str(i) + '.h5'
        classifier.save(filename=file_name, path=path)
    
    print("Re-training took {:.2f}s.".format(time.time() - start_time_re))

else:
    start_time = time.time()
    
    # Wrapper for generator
    # Because different learning rates does not seem to work with this wrapper, we do it manually.
    
    generator = KerasDataGenerator(train_generator, x_train.shape[0], batch_size)
    
    print("## Stage 1 ##")
    # trainer = AdversarialTrainer(classifier, attack, ratio=adv_ratio)
    # trainer.fit_generator(generator=generator, nb_epochs=epochs1, verbose=VERBOSITY, callbacks=callbacks, initial_epoch=initial_epoch, use_multiprocessing=use_multiprocessing)  
    # classifier.save(filename='stage1.h5', path=path)
    
    print("## Stage 2 ##")
    model2 = load_model('stage1.h5')
    model2.compile(optimizer=keras.optimizers.SGD(lr=lr2, momentum=momentum, decay=0.0), loss=loss, metrics=metrics)
    classifier2 = KerasClassifier(model=model2, clip_values=(min_, max_))
    attack2 = ProjectedGradientDescent(classifier=classifier2, norm=norm, eps=eps, eps_step=step_size, max_iter=steps, targeted=targeted, batch_size=batch_size)
    
    trainer2 = AdversarialTrainer(classifier2, attack2, ratio=adv_ratio)
    trainer2.fit_generator(generator=generator, nb_epochs=epochs2, verbose=VERBOSITY, callbacks=callbacks, initial_epoch=initial_epoch, use_multiprocessing=use_multiprocessing)  
    classifier2.save(filename='stage2.h5', path=path)
    
    print("## Stage 3 ##")
    model3 = load_model('stage2.h5')
    model3.compile(optimizer=keras.optimizers.SGD(lr=lr3, momentum=momentum, decay=0.0), loss=loss, metrics=metrics)
    classifier3 = KerasClassifier(model=model3, clip_values=(min_, max_))
    attack3 = ProjectedGradientDescent(classifier=classifier3, norm=norm, eps=eps, eps_step=step_size, max_iter=steps, targeted=targeted, batch_size=batch_size)
    
    trainer3 = AdversarialTrainer(classifier3, attack3, ratio=adv_ratio)
    trainer3.fit_generator(generator=generator, nb_epochs=epochs3, verbose=VERBOSITY, callbacks=callbacks, initial_epoch=initial_epoch, use_multiprocessing=use_multiprocessing)  
    
    classifier3.save(filename='stage3.h5', path=path)
    print("Training took {:.0f}s.".format(time.time() - start_time))

## Stage 1 ##
## Stage 2 ##
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
## Stage 3 ##
Training took 97050s.


In [20]:
test_size = len(y_test)
print("AFTER TRAINING:")
predictions = classifier3.predict(x_test[:test_size])
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test[:test_size], axis=1)) / len(y_test[:test_size])
print('Accuracy on benign test examples: {:.2f}%'.format(accuracy * 100))

predictions = classifier3.predict(x_test_adv)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test[:test_size], axis=1)) / len(y_test[:test_size])
print('Accuracy on OLD adversarial test examples: {:.2f}%'.format(accuracy * 100))

attack = ProjectedGradientDescent(classifier=classifier, norm=norm, eps=eps, eps_step=step_size, max_iter=steps, targeted=targeted, batch_size=batch_size)
x_test_adv = attack.generate(x=x_test[:test_size])
predictions = classifier3.predict(x_test_adv)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test[:test_size], axis=1)) / len(y_test[:test_size])
print('Accuracy on NEW adversarial test examples: {:.2f}%'.format(accuracy * 100))

AFTER TRAINING:
Accuracy on benign test examples: 70.80%
Accuracy on OLD adversarial test examples: 70.40%
Accuracy on NEW adversarial test examples: 70.40%
